In [11]:
from opik import configure 
from opik.integrations.langchain import OpikTracer 

configure() 
opik_tracer = OpikTracer(thread_id='user1') 

OPIK: Existing Opik clients will not use updated values for "url", "api_key", "workspace".
OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/aarsh/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In [12]:
import os
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI 
from langchain.tools import tool
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.orm import sessionmaker, declarative_base
from pprint import pprint
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [13]:
from datetime import datetime
from zoneinfo import ZoneInfo
# from langchain_core.tools import tool


In [14]:
llm = ChatOpenAI(base_url=os.getenv('BASE_URL'),
                 api_key=os.getenv('API_KEY'),
                 model=os.getenv('MODEL_NAME'), 
                 temperature=0.1)

In [15]:
# --- 1. Database Persistence (SQLAlchemy) ---
Base = declarative_base()

class ResolutionEntry(Base):
    __tablename__ = 'resolution_plans'
    id = Column(Integer, primary_key=True)
    resolution = Column(String(500))
    plan_json = Column(Text)

engine = create_engine('sqlite:///stride_resolutions.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

In [16]:
@tool
def save_resolution_to_db(resolution: str, plan_summary: str) -> str:
    """Saves the resolution and the generated milestones to the database."""
    session = Session()
    try:
        new_entry = ResolutionEntry(resolution=resolution, plan_json=plan_summary)
        session.add(new_entry)
        session.commit()
        return "Successfully saved to database."
    except Exception as e:
        session.rollback()
        return f"Error saving to database: {str(e)}"
    finally:
        session.close()

In [17]:
@tool
def current_time(tz: str = "Asia/Kolkata") -> str:
    """Get the current date/time in ISO format for a given IANA timezone."""
    return datetime.now(ZoneInfo(tz)).isoformat()


In [18]:
# --- 2. Structured Output Schema ---
class Milestone(BaseModel):
    period: int = Field(description="month number or week number")
    task: str = Field(description="The specific actionable milestone or habit")
    focus: str = Field(description="The primary objective for this unit")

class CalendarPlan(BaseModel):
    monthly_milestones: List[Milestone] = Field(description="monthly high-level goals")
    weekly_breakdown: Dict[int, List[Milestone]] = Field(description="each week focus areas for the all months where key is month and value is list of weekly milestones")
    daily_habits: List[str] = Field(description="Consistent daily actions to maintain momentum")

In [19]:
class FeasibilityVerdict(BaseModel):
    feasible: bool = Field(description='Feasibility')
    reason: str = Field(description='Reason why it is feasible or why not')


In [20]:
def check_resolution_feasibility(user_resolution: str):
    agent = create_agent(
        model=llm,
        system_prompt = """
                        You assess whether a user's goal is realistically achievable by an average human within the stated timeframe.
                        
                        Tool use:
                        - If the user gives a deadline date/time (e.g., "by tomorrow", "by Jan 31", "in 3 days"), you MUST call `current_time`
                          to anchor relative time (today/tomorrow/next week) and interpret the timeframe correctly.
                        - If the user gives an explicit duration (e.g., "in 30 days", "in 1 month"), you may skip calling the tool.
                        
                        Rules:
                        - If the goal or timeframe is missing/unclear, set feasible=false and explain what timeframe/info is needed.
                        - Assume typical starting skill and typical constraints unless the user provides specifics.
                        - Consider basic limits: time available, learning/skill acquisition rates, safety, and practicality.
                        - Be conservative: if it's doubtful, set feasible=false.
                        - Provide a short, concrete reason (1-3 sentences). Do not provide a plan.
                        - Output MUST match the provided schema exactly.
                        """,
        tools=[current_time],
        response_format=FeasibilityVerdict  # Ensures the agent returns the exact Pydantic schema
    )
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": f"My resolution is: {user_resolution}"}
        ]},
        config={"callbacks": [opik_tracer]}
                         )
    return result["structured_response"]

In [21]:
# --- 3. Agent Configuration ---
def run_resolution_agent(user_resolution: str):
    # Initialize the model (v1 uses improved content blocks)

    # create_agent is the v1 standard for building ReAct-style agents
    agent = create_agent(
        model=llm,
        tools=[save_resolution_to_db, current_time],
        system_prompt = """
                        You are a strategic planning agent. Your job is to turn a user's New Year's resolution into a structured calendar plan.
                        
                        Tool use (required):
                        - ALWAYS call the `current_time` tool at the start to get today's date/time (use timezone Asia/Kolkata unless the user specifies another).
                        - Use that date as the anchor for all dates you generate (e.g., “Week 1 starts on …”).
                        
                        Process:
                        1) Read the resolution and infer a realistic target timeframe if the user does not provide one (default: 4 weeks).
                        2) Create a structured calendar plan broken into weeks and days with clear milestones and measurable actions.
                        3) Include start_date (today) and end_date (based on timeframe) in the plan.
                        4) Keep the plan realistic for an average person and avoid unsafe guidance.
                        
                        Persistence:
                        - First, save the resolution and a brief summary of the plan using the provided saving tool.
                        - Then return the full structured calendar plan.
                        
                        Output rules:
                        - Return only the structured calendar plan (no extra commentary).
                        - Be specific: include dates, weekly milestones, and daily/recurring tasks.
                        """,
        response_format=CalendarPlan  # Ensures the agent returns the exact Pydantic schema
        
    )

    # Execution using the v1 invoke pattern
    # The 'messages' key is the standard input for v1 agents
    result = agent.invoke({
        "messages": [
            {"role": "user", "content": f"My resolution is: {user_resolution}"}
        ]},
        config={"callbacks": [opik_tracer]}
                         )

    return result["structured_response"]

In [22]:
# --- 4. Main Execution ---
if __name__ == "__main__":
    goal = "I want to be able to run 5 km by the end of the month"
    # goal = "I want to learn Python basics by tomorrow."
    print(f"Plan for: {goal}\n")
    resolution_feasibility = check_resolution_feasibility(goal)
    if resolution_feasibility.feasible:
        final_plan = run_resolution_agent(goal)
        
        print("--- Monthly Milestones ---")
        for m in final_plan.monthly_milestones:
            print(f"[{m.period}] {m.task} - Focus: {m.focus}")
    else:
        print(resolution_feasibility.reason)

    # print("\n--- Weekly Breakdown (Month 1) ---")
    # for w in final_plan.weekly_breakdown:
    #     print(f"[{w.period}] {w.task} - Focus: {w.focus}")
    
    # print("\n--- Daily Habits ---")
    # for habit in final_plan.daily_habits:


OPIK: Started logging traces to the "Default Project" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019beba7-58d4-705f-9c72-dfe6fd8de038&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


Plan for: I want to be able to run 5 km by the end of the month



OPIK: Filtering large LangGraph output (10918 chars) for thread display


--- Monthly Milestones ---
[1] Complete a 5 km run without stopping. - Focus: Build endurance and stamina.


In [23]:
final_plan.weekly_breakdown

{1: [Milestone(period=1, task='Alternate between walking and running intervals.', focus='Build a base level of endurance.'),
  Milestone(period=2, task='Run 1 km without stopping.', focus='Increase running distance.')],
 2: [Milestone(period=1, task='Run 1.5 km without stopping.', focus='Improve stamina.'),
  Milestone(period=2, task='Run 2 km without stopping.', focus='Increase running distance.')],
 3: [Milestone(period=1, task='Run 3 km without stopping.', focus='Build endurance.'),
  Milestone(period=2, task='Run 4 km without stopping.', focus='Increase running distance.')],
 4: [Milestone(period=1, task='Run 5 km without stopping.', focus='Achieve the goal.')]}

In [25]:
import os
import base64
from typing import List, Dict
from datetime import datetime
from zoneinfo import ZoneInfo
from dotenv import load_dotenv

# Opik and LangChain
from opik import configure 
from opik.integrations.langchain import OpikTracer 
from langchain_openai import ChatOpenAI 
from langchain.tools import tool
from langchain.agents import create_agent
from pydantic import BaseModel, Field

# Persistence
from sqlalchemy import create_engine, Column, Integer, String, Text
from sqlalchemy.orm import sessionmaker, declarative_base

# 1. Environment & Monitoring Setup
load_dotenv(override=True)
configure() 
opik_tracer = OpikTracer(thread_id='user_resolution_flow')

llm = ChatOpenAI(
    base_url=os.getenv('BASE_URL'),
    api_key=os.getenv('API_KEY'),
    model=os.getenv('MODEL_NAME'), 
    temperature=0.1
)

# 2. Database Persistence (SQLAlchemy)
Base = declarative_base()

class ResolutionEntry(Base):
    __tablename__ = 'resolution_plans'
    id = Column(Integer, primary_key=True)
    resolution = Column(String(500))
    plan_json = Column(Text)

engine = create_engine('sqlite:///stride_resolutions.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

# 3. Tools
@tool
def save_resolution_to_db(resolution: str, plan_summary: str) -> str:
    """Saves the resolution and the generated milestones to the database."""
    session = Session()
    try:
        new_entry = ResolutionEntry(resolution=resolution, plan_json=plan_summary)
        session.add(new_entry)
        session.commit()
        return "Successfully saved to database."
    except Exception as e:
        session.rollback()
        return f"Error saving to database: {str(e)}"
    finally:
        session.close()

@tool
def current_time(tz: str = "Asia/Kolkata") -> str:
    """Get the current date/time in ISO format for a given IANA timezone."""
    return datetime.now(ZoneInfo(tz)).isoformat()

# 4. Structured Output Schemas
class DailyTask(BaseModel):
    date: str = Field(description="The specific date (YYYY-MM-DD)")
    task: str = Field(description="The specific activity or rest planned for this day")

class Milestone(BaseModel):
    period: int = Field(description="month number or week number")
    task: str = Field(description="The specific actionable milestone")
    focus: str = Field(description="The primary objective for this unit")

class CalendarPlan(BaseModel):
    monthly_milestones: List[Milestone] = Field(description="monthly high-level goals")
    weekly_breakdown: Dict[int, List[Milestone]] = Field(description="Weekly focus areas")
    daily_schedule: List[DailyTask] = Field(description="An entry for EVERY single day from start to end")
    daily_habits: List[str] = Field(description="Consistent daily actions")

class FeasibilityVerdict(BaseModel):
    feasible: bool = Field(description='Feasibility')
    reason: str = Field(description='Reasoning')

# 5. Agent Logic
def check_resolution_feasibility(user_resolution: str):
    agent = create_agent(
        model=llm,
        system_prompt="""Assess if a goal is realistic. 
        If a deadline like 'tomorrow' or 'by Jan 31' is mentioned, call current_time.
        Assume typical human limits. Output FeasibilityVerdict schema.""",
        tools=[current_time],
        response_format=FeasibilityVerdict
    )
    result = agent.invoke(
        {"messages": [{"role": "user", "content": f"Resolution: {user_resolution}"}]},
        config={"callbacks": [opik_tracer]}
    )
    return result["structured_response"]

def run_resolution_agent(user_resolution: str):
    agent = create_agent(
        model=llm,
        tools=[save_resolution_to_db, current_time],
        system_prompt="""You are a strategic planner.
        1. Call current_time to get today's date.
        2. Create a DailyTask entry for EVERY SINGLE DAY from today until the goal deadline.
        3. Ensure specific, measurable tasks for each date.
        4. Save the plan to the DB using the tool.
        Return the CalendarPlan schema.""",
        response_format=CalendarPlan
    )
    result = agent.invoke(
        {"messages": [{"role": "user", "content": f"Resolution: {user_resolution}"}]},
        config={"callbacks": [opik_tracer]}
    )
    return result["structured_response"]

# 6. Calendar Link Utility
def generate_calendar_link(plan: CalendarPlan):
    """Generates a downloadable .ics link (Data URI)."""
    ics_content = ["BEGIN:VCALENDAR", "VERSION:2.0", "PRODID:-//StrideAI//EN"]
    for entry in plan.daily_schedule:
        clean_date = entry.date.replace("-", "")
        ics_content.extend([
            "BEGIN:VEVENT",
            f"DTSTART;VALUE=DATE:{clean_date}",
            f"SUMMARY:{entry.task}",
            f"DESCRIPTION:Generated by StrideAI",
            "END:VEVENT"
        ])
    ics_content.append("END:VCALENDAR")
    ics_string = "\n".join(ics_content)
    b64_ics = base64.b64encode(ics_string.encode()).decode()
    return f"data:text/calendar;base64,{b64_ics}"

# 7. Main Execution
if __name__ == "__main__":
    user_goal = "I want to be able to create my own llm models by the end of this year"
    print(f"--- Processing: {user_goal} ---\n")
    
    feasibility = check_resolution_feasibility(user_goal)
    
    if feasibility.feasible:
        plan = run_resolution_agent(user_goal)
        
        print("Daily Schedule Preview:")
        for day in plan.daily_schedule[:5]: # Show first 5 days
            print(f"  {day.date}: {day.task}")
        print("  ... (rest of the days generated) ...")
        
        cal_link = generate_calendar_link(plan)
        print("\n" + "="*50)
        print("✅ SUCCESS: PLAN GENERATED")
        print("Copy the link below and paste it into your browser to download:")
        print("-" * 50)
        print(cal_link)
        print("-" * 50)
    else:
        print(f"❌ Goal Unfeasible: {feasibility.reason}")

OPIK: Existing Opik clients will not use updated values for "url", "api_key", "workspace".
OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/aarsh/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


--- Processing: I want to be able to create my own llm models by the end of this year ---



OPIK: Filtering large LangGraph output (88865 chars) for thread display
OPIK: Filtering large LangGraph output (152328 chars) for thread display


Daily Schedule Preview:
  2026-01-23: Study machine learning basics for 2 hours.
  2026-01-24: Read about neural networks and their types.
  2026-01-25: Research large language models and their applications.
  2026-01-26: Understand transformer architecture.
  2026-01-27: Set up Python and necessary libraries.
  ... (rest of the days generated) ...

✅ SUCCESS: PLAN GENERATED
Copy the link below and paste it into your browser to download:
--------------------------------------------------
...
--------------------------------------------------
